# Very brief introduction to IGUA-df v.1.0-alpha.1

> NOTE: This version of IGUA-df still needs to be (1) cleaned up, as the code is messy and redundant (2) carefully checked for bugs. It can be used for preliminary analysis; however, it is not reliable, and the analysis may have to be rerun. 

## Installation 

This step may be a little clunky at the start. 

Install rust. 

Create a conda environment: `conda create igua-df python`

`conda activate igua-df`

Inside, nstall MMSeqs2: `conda install bioconda::mmseqs2`

For now, install IGUA in editable mode by following the instructions in [this commit](https://github.com/zellerlab/IGUA/commit/6ce03937bb7293ce3490771c523726db049fd0b3). To be more precise, clone the repo to IGUA-df and run 

```console
$ cd IGUA-df
$ python -m pip install --no-build-isolation -e .
```

There may be additional missing dependency messages. Please follow them. 

## Data analysis pipeline 

The first step is to take the raw and annotated data along with the DefenseFinder outputs and to run a pre-processing step that identifies all the files which will be fed to IGUA-df. To this end, I have written a quick function to help with the file detection; feel free to adapt it to your own needs. 

Please, note that the directory structure this function can work with is limited, as indicated in the docstring. 

In [ ]:
import os, subprocess
import rich 
from rich.console import Console
from rich.progress import Progress
import pandas as pd
from pathlib import Path
from typing import Optional

In [ ]:
def create_defense_finder_metadata(
    data_dir: str, 
    output_file: str,
    pattern: str = "*-defensefinder",
    progress: Optional[rich.progress.Progress] = None
) -> str:
    """
    Scan directory for defense-finder files and create a metadata TSV file.
    
    This function scans for strains with the following structure:
    data_dir/
    ├── strain1/
    │   ├── strain1.fa
    │   ├── strain1.faa  # Protein sequences
    │   ├── strain1.fna  # Gene sequences
    │   ├── strain1.gff
    │   └── strain1-defensefinder/
    │       ├── strain1_defense_finder_genes.tsv
    │       └── strain1_defense_finder_systems.tsv
    
    Args:
        data_dir: Base directory containing strain directories
        output_file: Path to output metadata TSV file
        pattern: Pattern to match defensefinder directories
        progress: Optional progress bar
        
    Returns:
        str: Path to created metadata file
    """
    console = Console() if progress is None else progress.console
    
    # List all subdirectories (strains)
    strain_dirs = [d for d in Path(data_dir).iterdir() if d.is_dir()]
    
    metadata_rows = []
    
    # progress 
    using_external_progress = progress is not None
    if not progress:
        progress = rich.progress.Progress(
            rich.progress.SpinnerColumn(),
            rich.progress.TextColumn("[bold blue]{task.description}"),
            rich.progress.BarColumn(),
            rich.progress.TextColumn("[bold]{task.completed}/{task.total}")
        )
    
    try:
        if not using_external_progress:
            progress.start()
            
        task = progress.add_task(f"[bold cyan]Scanning[/] {data_dir} for defense-finder data", total=len(strain_dirs))
        
        for strain_dir in strain_dirs:
            strain_id = strain_dir.name
            progress.update(task, description=f"[bold cyan]Scanning[/] strain: {strain_id}")
            
            # find genome and annotation files
            fa_files = list(strain_dir.glob(f"{strain_id}.fa"))
            gff_files = list(strain_dir.glob(f"{strain_id}.gff"))
            
            # find for protein and gene sequence files
            faa_files = list(strain_dir.glob(f"{strain_id}.faa"))
            fna_files = list(strain_dir.glob(f"{strain_id}.fna"))
            
            # find defensefinder directory
            defensefinder_dirs = list(strain_dir.glob(pattern))
            
            if not defensefinder_dirs:
                progress.console.print(f"[bold yellow]Warning:[/] No defensefinder directory found for strain {strain_id}")
                progress.update(task, advance=1)
                continue
                
            defensefinder_dir = defensefinder_dirs[0]
            
            # inside defensefinder, find TSV files
            systems_tsv = list(defensefinder_dir.glob(f"{strain_id}_defense_finder_systems.tsv"))
            genes_tsv = list(defensefinder_dir.glob(f"{strain_id}_defense_finder_genes.tsv"))
            
            # if all required files
            if (fa_files and gff_files and systems_tsv and genes_tsv):
                # metadata entry with optional protein/gene files
                # providing full paths
                metadata_entry = {
                    "strain_id": strain_id,
                    "systems_tsv": str(systems_tsv[0].absolute()),
                    "genes_tsv": str(genes_tsv[0].absolute()),
                    "gff_file": str(gff_files[0].absolute()),
                    "fasta_file": str(fa_files[0].absolute())
                }

                # Add protein and gene files if they exist                
                if faa_files:
                    metadata_entry["faa_file"] = str(faa_files[0].absolute())
                if fna_files:
                    metadata_entry["fna_file"] = str(fna_files[0].absolute())
                
                metadata_rows.append(metadata_entry)
            else:
                # Report missing files
                missing = []
                if not fa_files:
                    missing.append("FASTA")
                if not gff_files:
                    missing.append("GFF")
                if not systems_tsv:
                    missing.append("systems TSV")
                if not genes_tsv:
                    missing.append("genes TSV")
                progress.console.print(f"[bold yellow]Warning:[/] Missing files for strain {strain_id}: {', '.join(missing)}")
                
            progress.update(task, advance=1)
        
        # write metadata file
        if metadata_rows:
            df = pd.DataFrame(metadata_rows)
            df.to_csv(output_file, sep='\t', index=False)
            progress.console.print(f"[bold green]Created[/] metadata file with {len(metadata_rows)} strains: {output_file}")
        else:
            progress.console.print(f"[bold red]Error:[/] No valid strains found")
            
        return output_file
        
    finally:
        if not using_external_progress:
            progress.stop()

Upon running the function, a `strains_metadata.tsv` will be written. It will contain the paths to the `.fa`, `.gff`, `.faa`, `.fna`, `defense_finder_systems.tsv`, and `defense_finder_genes.tsv` along with some metadata about the strain ID (taken from the folder structure). Some of these files will be fed to IGUA internally. 

In [ ]:
data_dir = "/path/to/your/data/directory"  # Change to your data directory
test_run_directory = "/path/to/result/directory"  # Change to your desired working directory

create_defense_finder_metadata(
    data_dir=data_dir,  
    output_file=os.path.join(test_run_directory, "strains_metadata.tsv")
)

Next, from your console, navigate to the test run directory which should contain `strains_metadata.tsv`

```console
$ cd "/path/to/result/directory"
```

We are ready to run IGUA-df: 

```console
$ igua -i ./strains_metadata.tsv --defense-metadata ./strains_metadata.tsv --write-defense-systems defsysts 
```

Currently, the code is clunky, so the `strains_metadata.tsv` needs to be passed twice. This will be fixed in the future. 

Running this command will ideally result in a successful run of IGUA-df. The logger will indicate that final GCFs table was saved to `gcfs.tsv` in the same directory. In addition, the `--write-defense-systems defsysts` writes all the defense systems to `./defsysts`. It may be helpful to go back through the files for debugging/sanity-checking. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# quick look at the cluster sizes 
tsv_path = 'path/to/gcfs.tsv' 
gcfs_df = pd.read_csv(tsv_path, sep="\t")
gcfs_df['cluster_length'].hist(bins=100)
plt.xlabel('Cluster Length [bp]')
plt.ylabel('Frequency')
plt.show();
# plt.savefig('hist_cluster_length.pdf') 

In [ ]:
# quick look at column summaries 

for col in gcfs_df.columns:
    print(f"Column: {col}")
    print(gcfs_df[col].describe())
    print("\n")

On a final note, IGUA-df should be runnable also by providing single files as in

```console 
igua extract \
  --defense-systems-tsv /path/to/my_strain_defense_finder_systems.tsv \
  --defense-genes-tsv /path/to/my_strain_defense_finder_genes.tsv \
  --gff /path/to/my_strain.gff \
  --genome /path/to/my_strain.fa \
  --write-defense-systems /path/to/output/directory \
  --output /path/to/output/sequences.fasta
```

At this stage, this is unfortunately not possible. But it will be fixed in the future. 

## Limitations and known/potential bugs 

- currently, the defense system extraction functions do not filter by `activity` _i.e._ **they do not distinguish defense from anti-defense systems**
- I have not had a chance to go back and cross-referene the defense systems and whether they cluster sensibly (_e.g._ do subtypes cluster) 
- I still have a 1 GCF discrepancy between my old implementation (that had two processing steps, where defense system genomic sequence and protein sequence was external to IGUA-df processing). The old version converges to 92 GCFs while the new one writes 91 GCFs. I have not had a chance to dig into why this is the case yet. 
- I may have missed a potential bug so far -- I am relying on the more thorough validation step for these kinds of bugs. 